#MIT 6.036 Spring 2019: Lab 13#

This colab notebook provides an alternative space to complete [Lab 13](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week13/week13_lab/).

## <section>**Setup**</section>

Run the next code block to download and import the code for this lab.

In [6]:
#!rm -rf code_for_lab13* __MACOSX data .DS_Store
#!wget --quiet https://introml_oll.odl.mit.edu/6.036/static/labs/lab13/code_for_lab13.zip
#!unzip code_for_lab13.zip
#!mv code_for_lab13/* .

from non_parametric import auto_data_and_labels, auto_data_all, one_hot, standard, raw, auto_xval

from IPython.display import clear_output
# clear_output()

# 1) Basic operation

Look at the code directory and open the file `non_parametric.py`. You will see an implementation of the recursive (greedy) decision-tree building algorithm described in lecture, and an implementation of a nearest neighbor classifier.

We will be running this algorithm on the Auto-MPG dataset that we used in Lab 3 and HW 3; you should look back at the description on the Lab 3 and your results from HW 3 re-familiarize yourself with the data. In the `non_parametric.py` file, towards the end is the code from HW 3 that we used to read the data into Numpy matrices. The one difference is that we are reading the individual points into the *rows* of the matrices rather than the columns as we did before.

Find the place where `features` are defined. This is as before, you can choose there whether a feature is kept in its `raw` form, converted to `standard` or to `one_hot`. Note that the call


```
auto_data, auto_labels = auto_data_and_labels(auto_data_all, features)
```

will create the data and labels matrices. Crucially, it will also print which feature index corresponds to which original feature. For example:


```
0 cylinders 3.0 one_hot
1 cylinders 4.0 one_hot
2 cylinders 5.0 one_hot
3 cylinders 6.0 one_hot
4 cylinders 8.0 one_hot
5 displacement std
6 horsepower std
7 weight std
8 acceleration std
9 origin 1.0 one_hot
10 origin 2.0 one_hot
11 origin 3.0 one_hot
```


The first number is the feature index, which plays a prominent role in the decision-tree construction. Remember that the one_hot transformation produces a number of features with values either 0 or 1 corresponding to a discrete value of the original feature, so:



```
2 cylinders 5.0 one_hot
```

means that feature index 2 corresponds to 1 when cylinders is equal to 5.0 and 0 otherwise. In the default setting all the features are raw. Note that by default we're ignoring model name and model year, as we discussed in Lab 3.

Run the following code block to try out the above.

In [7]:
features = [('cylinders', one_hot),
            ('displacement', standard),
            ('horsepower', standard),
            ('weight', standard),
            ('acceleration', standard),
            ## Drop model_year by default
            ## ('model_year', raw),
            ('origin', one_hot)]

auto_data, auto_labels = auto_data_and_labels(auto_data_all, features)

avg and std {'displacement': (388.3482142857143, 302.0458123396403), 'horsepower': (509.3545918367347, 333.6521151716361), 'weight': (2977.5841836734694, 848.3184465698365), 'acceleration': (15.541326530612228, 2.7553429127509963)}
entries in one_hot field {'cylinders': [3.0, 4.0, 5.0, 6.0, 8.0], 'origin': [1.0, 2.0, 3.0]}
0 cylinders 3.0 one_hot
1 cylinders 4.0 one_hot
2 cylinders 5.0 one_hot
3 cylinders 6.0 one_hot
4 cylinders 8.0 one_hot
5 displacement std
6 horsepower std
7 weight std
8 acceleration std
9 origin 1.0 one_hot
10 origin 2.0 one_hot
11 origin 3.0 one_hot


Two useful functions defined in the file are:



*   `auto_test(pct=0.25)` splits the auto data (randomly) into training and test sets of fractions `1-pct` and `pct`, respectively, and runs the decision tree construction algorithm on the training data and evaluates the resulting tree on the test set. By default, it prints the tree to depth 5 (see the definition of `dt_eval`) and prints (and returns) the accuracy
*   `auto_xval(k=10)` does k-fold cross validation. It also calls `dt_eval` and print the trees and accuracy. At the end the function returns the average accuracy across the runs.

A tree (that corresponds to the feature definitions above) might look like this (note that `n` is the number of remaining points in the subtree):



```
Feat 7 < 0.0476422698221 [ n= 294.0 ]
   Feat 7 < -0.251184192133 [ n= 171.0 ]
     Feat 2 < 0.5 [ n= 142.0 ]
       Feat 8 < -0.940473332238 [ n= 8.0 ]
         => 1.00 [ n= 4.0 ]
         => 0.00 [ n= 4.0 ]
       Feat 7 < -0.887737602216 [ n= 134.0 ]
         => 1.00 [ n= 74.0 ]
         Feat 7 < -0.442150215158 [ n= 60.0 ]
           Feat 8 < 0.837889708284 [ n= 42.0 ]
             Depth > 5
             Depth > 5
           => 1.00 [ n= 18.0 ]
     Feat 8 < 0.239053174231 [ n= 29.0 ]
       Feat 5 < -0.628541123663 [ n= 17.0 ]
         Feat 5 < -0.853672534932 [ n= 11.0 ]
           => 0.00 [ n= 4.0 ]
           Feat 6 < 1.35064454164 [ n= 7.0 ]
             Depth > 5
             Depth > 5
         => 0.00 [ n= 6.0 ]
       Feat 8 < 0.583837845352 [ n= 12.0 ]
         => 1.00 [ n= 4.0 ]
         Feat 6 < 0.961017160039 [ n= 8.0 ]
           => 1.00 [ n= 2.0 ]
           Feat 6 < 1.14084518232 [ n= 6.0 ]
             Depth > 5
             Depth > 5
   Feat 8 < 1.54560561217 [ n= 123.0 ]
     Feat 8 < 0.638277530266 [ n= 117.0 ]
       => 0.00 [ n= 102.0 ]
       Feat 5 < -0.203108971485 [ n= 15.0 ]
         => 0.00 [ n= 14.0 ]
         => 1.00 [ n= 1.0 ]
     Feat 8 < 2.36220088587 [ n= 6.0 ]
       Feat 3 < 0.5 [ n= 4.0 ]
         => 0.00 [ n= 3.0 ]
         => 1.00 [ n= 1.0 ]
       => 1.00 [ n= 2.0 ]
```


The printing is done by recursively exploring the tree, printing the test at a node (`Feat i < th [ n= n ]`, then going to the left child (the Yes node) first and then the right child (the No node). When it gets to a leaf in the tree, it prints '=> p [ n= n ]', where *p* is the prediction (probability of positive label).

Make sure that you can read the tree. Note: Why are some of the thresholds negative values?

###1A) What do the classes that we are trying to predict in this problem mean? (Review the description in [Lab 3](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week3/week3_lab/))

ANS) 

The class we are trying to predict in this problem is the whether the MPG (fuel efficiency) of the car is above or below a certain threshold.

###1B) Run a 10-fold cross validation with the one-hot and standardized features (run the next code block to do this). This will print 10 trees, each with an accuracy, then an overall accuracy.



*   Look back to [HW 3](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week3/week3_homework/) and find the best accuracy we got with Average Perceptron for those features and compare.
*   Look at the tests being used at the top layers of the trees, what features are the tests splitting on?
*   Look at the leaves of the trees; what does the float between 0 and 1 represent? How do you decide what class to output?



Ans) The first feature the test is splitting on is 4 cylinders, followed by displacement and then acceleration.

The float between 0 and 1 for each leaf node represents the probability of the positive class for the data points that reach that leaf.

In [8]:
features = [('cylinders', one_hot),
            ('displacement', standard),
            ('horsepower', standard),
            ('weight', standard),
            ('acceleration', standard),
            ## Drop model_year by default
            ## ('model_year', raw),
            ('origin', one_hot)]

auto_data, auto_labels = auto_data_and_labels(auto_data_all, features)

auto_xval(auto_data, auto_labels, k=10)

avg and std {'displacement': (388.3482142857143, 302.0458123396403), 'horsepower': (509.3545918367347, 333.6521151716361), 'weight': (2977.5841836734694, 848.3184465698365), 'acceleration': (15.541326530612228, 2.7553429127509963)}
entries in one_hot field {'cylinders': [3.0, 4.0, 5.0, 6.0, 8.0], 'origin': [1.0, 2.0, 3.0]}
0 cylinders 3.0 one_hot
1 cylinders 4.0 one_hot
2 cylinders 5.0 one_hot
3 cylinders 6.0 one_hot
4 cylinders 8.0 one_hot
5 displacement std
6 horsepower std
7 weight std
8 acceleration std
9 origin 1.0 one_hot
10 origin 2.0 one_hot
11 origin 3.0 one_hot
 Feat 1 < 0.5 [ n= 313.0 ]
   Feat 5 < -0.6285411236631759 [ n= 158.0 ]
     Feat 8 < -0.8497405240477289 [ n= 21.0 ]
       => 1.00 [ n= 5.0 ]
       Feat 8 < 0.9467690781120286 [ n= 16.0 ]
         Feat 7 < -0.1285887205618948 [ n= 13.0 ]
           => 0.00 [ n= 5.0 ]
           Feat 7 < -0.024264689465024896 [ n= 8.0 ]
             Depth > 5
             Depth > 5
         => 1.00 [ n= 3.0 ]
     Feat 7 < 0.04587406

0.44261603375527425

###1C) Use the following small two-feature set in the next code block and repeat the process in 1B. Draw the top two layers of the tree, what do they represent?

ANS) Each non-leaf node represents a decision rule based on one of the features, along with a threshold value. The tree partitions the data into smaller subsets based on the threshold values as it goes down the tree. The leaf nodes are the final predictions for the data points that reach them.

In [9]:
# A small feature set
features = [('weight', standard),
            ('origin', one_hot)]

auto_data, auto_labels = auto_data_and_labels(auto_data_all, features)

auto_xval(auto_data, auto_labels, k=10)

avg and std {'weight': (2977.5841836734694, 848.3184465698365)}
entries in one_hot field {'origin': [1.0, 2.0, 3.0]}
0 weight std
1 origin 1.0 one_hot
2 origin 2.0 one_hot
3 origin 3.0 one_hot
 Feat 0 < -0.25118419213335774 [ n= 313.0 ]
   Feat 0 < -0.8877376022159538 [ n= 151.0 ]
     Feat 0 < -1.0067966659536245 [ n= 79.0 ]
       => 1.00 [ n= 53.0 ]
       Feat 0 < -1.0056178633423605 [ n= 26.0 ]
         => 0.00 [ n= 1.0 ]
         => 1.00 [ n= 25.0 ]
     Feat 0 < -0.38144188067803714 [ n= 72.0 ]
       Feat 0 < -0.8847905956877937 [ n= 57.0 ]
         => 0.00 [ n= 1.0 ]
         Feat 0 < -0.8240822612076942 [ n= 56.0 ]
           => 1.00 [ n= 9.0 ]
           Feat 0 < -0.8181882481513739 [ n= 47.0 ]
             Depth > 5
             Depth > 5
       Feat 3 < 0.5 [ n= 15.0 ]
         => 1.00 [ n= 13.0 ]
         => 0.50 [ n= 2.0 ]
   Feat 0 < 0.32937609391419514 [ n= 162.0 ]
     Feat 1 < 0.5 [ n= 53.0 ]
       Feat 0 < 0.20913822756526035 [ n= 18.0 ]
         Feat 0 < -0.110317

0.41580655631288543

###1D) Does it change anything if we use `raw` for origin and/or standardize weight (run the next code block to do this)? Explain.

If you use raw for the origin feature, it will be treated as a numeric feature and its raw values will be used directly in the model. This is problematic, because origin is a categorical split, meaning using raw values will imply an ordering of the categories.

Standardizing the weight is useful because the scale of the weight feature is very different from the scale of other features in the data. Tt helps prevent the weight feature from dominating the model.

In [10]:
# The choice of feature processing for each feature, mpg is always raw and
# does not need to be specified.  Other choices are standard and one_hot.
# 'name' is not numeric and would need a different encoding.
features = [('weight', raw),
            ('origin', raw)]

auto_data, auto_labels = auto_data_and_labels(auto_data_all, features)

auto_xval(auto_data, auto_labels, k=10)

avg and std {}
entries in one_hot field {}
0 weight raw
1 origin raw
 Feat 0 < 2764.5 [ n= 313.0 ]
   Feat 0 < 2224.5 [ n= 151.0 ]
     Feat 0 < 2123.5 [ n= 79.0 ]
       => 1.00 [ n= 53.0 ]
       Feat 0 < 2124.5 [ n= 26.0 ]
         => 0.00 [ n= 1.0 ]
         => 1.00 [ n= 25.0 ]
     Feat 0 < 2654.0 [ n= 72.0 ]
       Feat 0 < 2227.0 [ n= 57.0 ]
         => 0.00 [ n= 1.0 ]
         Feat 0 < 2278.5 [ n= 56.0 ]
           => 1.00 [ n= 9.0 ]
           Feat 0 < 2283.5 [ n= 47.0 ]
             Depth > 5
             Depth > 5
       Feat 1 < 2.5 [ n= 15.0 ]
         => 1.00 [ n= 13.0 ]
         => 0.50 [ n= 2.0 ]
   Feat 0 < 3257.0 [ n= 162.0 ]
     Feat 1 < 1.5 [ n= 53.0 ]
       Feat 0 < 3016.5 [ n= 35.0 ]
         Feat 0 < 2996.5 [ n= 18.0 ]
           Feat 0 < 2909.0 [ n= 16.0 ]
             Depth > 5
             Depth > 5
           => 1.00 [ n= 2.0 ]
         => 0.00 [ n= 17.0 ]
       Feat 0 < 3155.0 [ n= 18.0 ]
         Feat 0 < 2884.0 [ n= 14.0 ]
           => 0.00 [ n= 5.0 ]


0.41838688737422913

###1E) When might we want to use a decision tree classifier as opposed to an averaged perceptron or neural net classifier?

ANS) 
Its better to use decision trees when datasets are small, the data is missing some values, or when the data is a mix of categorical and non-categorical data.